# Analyse complète des variables BAAC

## Objectif
Pour chaque variable, déterminer :
- **Usage direct** : utilisable telle quelle
- **Agrégation** : comment agréger quand plusieurs lignes
- **Feature engineering** : nouvelles features à créer
- **Data leakage** : disponible en temps réel ou non

## Légende
- 🟢 **Temps réel** : disponible immédiatement (appel secours)
- 🟡 **Partiellement** : parfois disponible rapidement
- 🔴 **Post-accident** : connu après enquête/constat
- ⚠️ **TARGET** : ne pas utiliser comme feature

---
# 1. TABLE CARACTÉRISTIQUES (15 colonnes)

**Granularité** : 1 ligne par accident

| Variable | Description | Type | Temps réel | Usage direct | Feature engineering |
|----------|-------------|------|------------|--------------|---------------------|
| `Num_Acc` | Identifiant | ID | - | ❌ Supprimer | Clé de jointure uniquement |
| `jour` | Jour du mois | Num | 🟢 | ✅ | → `is_fin_mois` (>25) |
| `mois` | Mois | Cat | 🟢 | ✅ | → `saison`, `vacances_scolaires` |
| `an` | Année | Num | 🟢 | ❌ (une seule année) | - |
| `hrmn` | Heure:minute | Num | 🟢 | ⚠️ Transformer | → `heure`, `minute`, `periode_journee`, `heure_pointe` |
| `lum` | Luminosité | Cat | 🟢 | ✅ | → `is_nuit` (3,4,5) |
| `dep` | Département | Cat | 🟢 | ⚠️ Trop de modalités | → Regrouper en régions ou utiliser lat/long |
| `com` | Commune | Cat | 🟢 | ❌ Trop granulaire | Enrichir avec données externes (population, densité) |
| `agg` | Agglomération | Cat | 🟢 | ✅ | Très important (rural vs urbain) |
| `int` | Type intersection | Cat | 🟢 | ✅ | → `is_intersection` (!=1), `is_giratoire` (==6) |
| `atm` | Météo | Cat | 🟢 | ✅ | → `meteo_degradee` (2-9), `visibilite_reduite` (5,7) |
| `col` | Type collision | Cat | 🟢 | ✅ **TRÈS PRÉDICTIF** | → `is_frontale` (1), `is_sans_collision` (7) |
| `adr` | Adresse | Texte | 🟢 | ❌ Supprimer | Non exploitable directement |
| `lat` | Latitude | Num | 🟢 | ✅ | → Clustering géographique, distance hôpital |
| `long` | Longitude | Num | 🟢 | ✅ | → Idem |

### Détail des features engineering CARACTÉRISTIQUES

```python
# Temporel
df['heure'] = df['hrmn'].astype(str).str.zfill(4).str[:2].astype(int)
df['minute'] = df['hrmn'].astype(str).str.zfill(4).str[2:].astype(int)
df['periode_journee'] = pd.cut(df['heure'], bins=[0,6,12,18,24], labels=['nuit','matin','aprem','soir'])
df['heure_pointe'] = df['heure'].isin([7,8,9,17,18,19]).astype(int)
df['jour_semaine'] = date.dayofweek
df['is_weekend'] = df['jour_semaine'].isin([5,6]).astype(int)
df['nuit_weekend'] = (df['is_nuit'] & df['is_weekend']).astype(int)  # TRÈS À RISQUE

# Luminosité
df['is_nuit'] = df['lum'].isin([3,4,5]).astype(int)
df['eclairage_absent'] = df['lum'].isin([3,4]).astype(int)  # Nuit sans/avec éclairage non allumé

# Météo
df['meteo_degradee'] = (~df['atm'].isin([-1, 1])).astype(int)
df['visibilite_reduite'] = df['atm'].isin([5, 7]).astype(int)  # Brouillard, éblouissant
df['chaussee_glissante'] = df['atm'].isin([2,3,4]).astype(int)  # Pluie, neige

# Collision
df['is_frontale'] = (df['col'] == 1).astype(int)  # TRÈS MORTEL
df['is_sans_collision'] = (df['col'] == 7).astype(int)  # Véhicule seul
df['multi_vehicules'] = df['col'].isin([4,5]).astype(int)  # Chaîne ou multiples

# Intersection
df['is_intersection'] = (df['int'] != 1).astype(int)
df['is_giratoire'] = (df['int'] == 6).astype(int)
```

---
# 2. TABLE LIEUX (18 colonnes)

**Granularité** : Peut avoir plusieurs lignes par accident (intersection multi-routes)
**Solution** : `drop_duplicates(subset='Num_Acc')` ou agréger

| Variable | Description | Type | Temps réel | Usage direct | Feature engineering |
|----------|-------------|------|------------|--------------|---------------------|
| `Num_Acc` | Identifiant | ID | - | Jointure | - |
| `catr` | Catégorie route | Cat | 🟢 | ✅ **IMPORTANT** | → `is_autoroute` (1), `is_nationale` (2), `route_rapide` (1,2) |
| `voie` | Numéro route | Texte | 🟢 | ❌ Trop granulaire | - |
| `v1`, `v2` | Indice route | Texte | 🟢 | ❌ | - |
| `circ` | Régime circulation | Cat | 🟢 | ✅ | → `bidirectionnelle` (2), `chaussees_separees` (3) |
| `nbv` | Nb voies | Num | 🟢 | ✅ | → `route_large` (>2) |
| `vosp` | Voie réservée | Cat | 🟢 | ✅ | → `has_piste_cyclable` (1,2) |
| `prof` | Profil (pente) | Cat | 🟢 | ✅ | → `is_pente` (2,3,4) |
| `pr`, `pr1` | Point repère | Num | 🟢 | ❌ | - |
| `plan` | Tracé (courbe) | Cat | 🟢 | ✅ | → `is_virage` (2,3,4) |
| `lartpc` | Largeur terre-plein | Num | 🟢 | ⚠️ Beaucoup de NaN | → `has_terre_plein` (>0) |
| `larrout` | Largeur chaussée | Num | 🟢 | ✅ | Normaliser, imputer |
| `surf` | État surface | Cat | 🟢 | ✅ **IMPORTANT** | → `surface_degradee` (2-9), `verglas_neige` (5,7) |
| `infra` | Infrastructure | Cat | 🟢 | ✅ | → `is_tunnel` (1), `is_pont` (2) |
| `situ` | Situation | Cat | 🟢 | ✅ | → `hors_chaussee` (2-8) |
| `vma` | Vitesse max | Num | 🟢 | ✅ **TRÈS IMPORTANT** | → `zone_30` (<=30), `haute_vitesse` (>=110) |

### Détail des features engineering LIEUX

```python
# Type de route
df['is_autoroute'] = (df['catr'] == 1).astype(int)
df['is_nationale'] = (df['catr'] == 2).astype(int)
df['is_departementale'] = (df['catr'] == 3).astype(int)
df['is_communale'] = (df['catr'] == 4).astype(int)
df['route_rapide'] = df['catr'].isin([1, 2]).astype(int)

# Caractéristiques route
df['is_virage'] = df['plan'].isin([2, 3, 4]).astype(int)
df['is_pente'] = df['prof'].isin([2, 3, 4]).astype(int)
df['bidirectionnelle'] = (df['circ'] == 2).astype(int)  # Plus de risque frontal
df['chaussees_separees'] = (df['circ'] == 3).astype(int)  # Plus sécurisé

# Surface
df['surface_mouillée'] = df['surf'].isin([2, 3, 4]).astype(int)
df['surface_glissante'] = df['surf'].isin([5, 6, 7, 8]).astype(int)  # Neige, boue, verglas, huile

# Vitesse - TRÈS PRÉDICTIF
df['zone_30'] = (df['vma'] <= 30).astype(int)
df['zone_50'] = (df['vma'] == 50).astype(int)
df['zone_80_90'] = df['vma'].isin([80, 90]).astype(int)
df['haute_vitesse'] = (df['vma'] >= 110).astype(int)

# Infrastructure spéciale
df['infra_dangereuse'] = df['infra'].isin([1, 2, 3]).astype(int)  # Tunnel, pont, bretelle
```

---
# 3. TABLE VÉHICULES (11 colonnes)

**Granularité** : Plusieurs lignes par accident (1 par véhicule impliqué)
**Solution** : Agréger par `Num_Acc`

| Variable | Description | Type | Temps réel | Usage direct | Feature engineering |
|----------|-------------|------|------------|--------------|---------------------|
| `Num_Acc` | Identifiant | ID | - | Jointure | - |
| `id_vehicule` | ID véhicule | ID | - | Jointure usagers | `nb_vehicules = nunique()` |
| `num_veh` | ID alphanum | ID | - | ❌ | - |
| `senc` | Sens circulation | Cat | 🟡 | ❌ Peu utile | → `sens_opposes` si 2 véhicules |
| `catv` | **Catégorie véhicule** | Cat | 🟢 | ✅ **TRÈS IMPORTANT** | Voir détail ci-dessous |
| `obs` | Obstacle fixe | Cat | 🟡 | Agréger | → `has_obstacle_fixe`, `type_obstacle` |
| `obsm` | Obstacle mobile | Cat | 🟡 | Agréger | → `has_pieton_heurte` (1) |
| `choc` | Point de choc | Cat | 🔴 | Agréger | → `choc_frontal` (1), `choc_lateral` (7,8) |
| `manv` | Manœuvre | Cat | 🔴 | Agréger | → `depassement` (17,18), `contresens` (5) |
| `motor` | Motorisation | Cat | 🟡 | Agréger | → `has_electrique` (3) |
| `occutc` | Occupants TC | Num | 🟡 | Agréger | → `sum()` si transport en commun |

### Détail CATV - Catégorie de véhicule (CRUCIAL)

```python
# Regroupements par vulnérabilité/dangerosité
PIETONS_VELOS = [1, 80]  # Bicyclette, VAE
EDP = [50, 60]  # Trottinettes (avec/sans moteur)
CYCLOMOTEURS = [2, 30]  # <50cm3
MOTOS_LEGERES = [31, 32]  # 50-125cm3
MOTOS_LOURDES = [33, 34]  # >125cm3
VOITURES = [7]  # VL seul
UTILITAIRES = [10]  # VU 1.5-3.5T
POIDS_LOURDS = [13, 14, 15, 16, 17]  # PL et tracteurs
TRANSPORT_COMMUN = [37, 38]  # Bus, autocar
AGRICOLE = [21]  # Tracteur agricole

# Agrégation par accident
vehicules_agg = vehicules.groupby('Num_Acc').agg(
    nb_vehicules=('id_vehicule', 'nunique'),
    
    # Présence de types vulnérables (TRÈS MORTEL)
    has_velo=('catv', lambda x: x.isin([1, 80]).any()),
    has_edp=('catv', lambda x: x.isin([50, 60]).any()),
    has_moto=('catv', lambda x: x.isin([31, 32, 33, 34]).any()),
    has_cyclomoteur=('catv', lambda x: x.isin([2, 30]).any()),
    
    # Présence de types lourds (DANGEREUX)
    has_poids_lourd=('catv', lambda x: x.isin([13, 14, 15, 16, 17]).any()),
    has_bus=('catv', lambda x: x.isin([37, 38]).any()),
    
    # Indicateurs de risque
    has_2_roues=('catv', lambda x: x.isin([1, 2, 30, 31, 32, 33, 34, 50, 60, 80]).any()),
    collision_leger_lourd=('catv', lambda x: 
        x.isin([1, 2, 7, 30, 31, 32, 33, 34, 50, 60, 80]).any() and 
        x.isin([13, 14, 15, 16, 17, 37, 38]).any()),  # TRÈS MORTEL
    
    # Obstacles
    has_obstacle_fixe=('obs', lambda x: (x > 0).any()),
    obstacle_arbre=('obs', lambda x: (x == 2).any()),  # Arbre = mortel
    has_pieton_heurte=('obsm', lambda x: (x == 1).any()),
    
    # Choc
    choc_frontal=('choc', lambda x: (x == 1).any()),
    choc_tonneaux=('choc', lambda x: (x == 9).any()),  # Très grave
    
    # Manœuvres dangereuses
    manv_depassement=('manv', lambda x: x.isin([17, 18]).any()),
    manv_contresens=('manv', lambda x: (x == 5).any()),
    manv_evitement=('manv', lambda x: (x == 21).any())
).reset_index()
```

---
# 4. TABLE USAGERS (16 colonnes)

**Granularité** : Plusieurs lignes par accident (1 par personne impliquée)
**Solution** : Agréger par `Num_Acc` OU modéliser par usager

| Variable | Description | Type | Temps réel | Usage direct | Feature engineering |
|----------|-------------|------|------------|--------------|---------------------|
| `Num_Acc` | Identifiant | ID | - | Jointure | - |
| `id_usager` | ID usager | ID | - | - | `nb_usagers = count()` |
| `id_vehicule` | ID véhicule | ID | - | Jointure | - |
| `place` | Place dans véhicule | Cat | 🔴 | - | → `conducteur` (1), `passager_avant` (2) |
| `catu` | Catégorie usager | Cat | 🟡 | Agréger | → `nb_pietons`, `nb_conducteurs` |
| ⚠️ `grav` | **GRAVITÉ = TARGET** | Cat | ❌ | **NE PAS UTILISER** | C'est ce qu'on prédit ! |
| `sexe` | Sexe | Cat | 🔴 | Agréger | → `pct_hommes`, `has_femme` |
| `an_nais` | Année naissance | Num | 🔴 | → Âge | → `age_moyen`, `has_enfant`, `has_senior` |
| `trajet` | Motif trajet | Cat | 🔴 | Agréger | → `trajet_pro` (4), `trajet_loisir` (5) |
| `secu1/2/3` | Équipement sécu | Cat | 🔴 | Agréger | → `taux_ceinture`, `taux_casque` |
| `locp` | Localisation piéton | Cat | 🟡 | Agréger | → `pieton_hors_passage` (1,2) |
| `actp` | Action piéton | Cat | 🔴 | Agréger | → `pieton_traversant` (3) |
| `etatp` | État piéton | Cat | 🔴 | Agréger | → `pieton_seul`, `pieton_groupe` |

### Détail agrégation USAGERS

```python
usagers_agg = usagers.groupby('Num_Acc').agg(
    # Comptages
    nb_usagers=('id_usager', 'count'),
    nb_vehicules_avec_usagers=('id_vehicule', 'nunique'),
    
    # Catégories d'usagers
    nb_conducteurs=('catu', lambda x: (x == 1).sum()),
    nb_passagers=('catu', lambda x: (x == 2).sum()),
    nb_pietons=('catu', lambda x: (x == 3).sum()),
    has_pieton=('catu', lambda x: (x == 3).any()),
    
    # Démographie
    pct_hommes=('sexe', lambda x: (x == 1).mean()),
    age_moyen=('an_nais', lambda x: (2024 - x.replace(-1, np.nan)).mean()),
    age_min=('an_nais', lambda x: (2024 - x.replace(-1, np.nan)).min()),
    age_max=('an_nais', lambda x: (2024 - x.replace(-1, np.nan)).max()),
    has_enfant=('an_nais', lambda x: ((2024 - x) < 18).any()),  # <18 ans
    has_jeune=('an_nais', lambda x: ((2024 - x).between(18, 25)).any()),  # 18-25 ans À RISQUE
    has_senior=('an_nais', lambda x: ((2024 - x) > 65).any()),  # >65 ans
    
    # Équipements (⚠️ souvent post-accident)
    taux_ceinture=('secu1', lambda x: (x == 1).mean()),
    has_sans_ceinture=('secu1', lambda x: (x == 0).any()),
    taux_casque=('secu1', lambda x: (x == 2).mean()),
    
    # Piétons
    pieton_traversant=('actp', lambda x: (x == 3).any()),
    pieton_hors_passage=('locp', lambda x: x.isin([1, 2]).any()),
).reset_index()
```

### ⚠️ Attention Data Leakage

Beaucoup de variables usagers sont connues **après l'accident** :
- `secu1/2/3` : équipement vérifié par les forces de l'ordre
- `place` : position dans le véhicule
- `trajet` : reconstitué après
- `actp`, `locp` : action/localisation du piéton

Pour un modèle **temps réel**, ces features ne sont pas disponibles.

---
# 5. Choix de la granularité

## Option A : 1 ligne par ACCIDENT (~54 000 lignes)

**Target** : `mortel = (grav == 2).any()` → L'accident a-t-il fait au moins un mort ?

**Avantages** :
- Correspond au use case "priorisation secours"
- Données indépendantes (pas de corrélation intra-accident)
- Plus simple à interpréter

**Inconvénients** :
- Perte d'information (agrégation des véhicules/usagers)
- Classes très déséquilibrées (~5.9% mortels)

---

## Option B : 1 ligne par VÉHICULE (~70 000 lignes)

**Target** : `mortel = (grav == 2).any()` par véhicule → Ce véhicule a-t-il un occupant mort ?

**Avantages** :
- On garde les infos véhicule directement
- Peut prédire quel véhicule est le plus touché

**Inconvénients** :
- Un même accident apparaît plusieurs fois → **données corrélées**
- Problème de validation (split doit être par accident)
- Plus complexe

---

## Option C : 1 ligne par USAGER (~125 000 lignes)

**Target** : `mortel = (grav == 2)` → Cet usager est-il décédé ?

**Avantages** :
- On peut utiliser âge, sexe, place dans véhicule...
- Plus de données
- Peut identifier les profils à risque

**Inconvénients** :
- Données très corrélées (même accident, même véhicule)
- Nécessite **GroupKFold** pour la validation
- Classes encore plus déséquilibrées (~2.7% décès)

---
# 6. Ma recommandation

## Pour un projet Simplon d'1 semaine

Je recommande **Option A (par accident)** avec une approche progressive :

### Modèle 1 : Baseline "circonstances" (ce que tu as fait)
- Features : CARACT + LIEUX uniquement
- Sans `col` pour éviter leakage débat
- ROC-AUC attendu : ~0.74

### Modèle 2 : + Type collision
- Ajouter `col`
- ROC-AUC attendu : ~0.76

### Modèle 3 : + Véhicules agrégés
- Ajouter features véhicules (catv, obstacles...)
- ROC-AUC attendu : ~0.78-0.80

### Modèle 4 (bonus) : + Usagers agrégés
- Ajouter nb_usagers, has_pieton, age_moyen...
- ROC-AUC attendu : ~0.80-0.82

---

## Pour aller plus loin (si temps)

**Option C (par usager)** serait intéressante pour :
- Prédire si un individu spécifique va mourir
- Utiliser l'âge, le sexe, la position...
- Mais nécessite `GroupKFold` pour éviter le leakage

```python
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(n_splits=5)
for train_idx, test_idx in gkf.split(X, y, groups=df['Num_Acc']):
    # Les usagers du même accident sont toujours ensemble
```

---
# 7. Synthèse des features par importance estimée

## 🔥 Features TRÈS prédictives (à inclure absolument)
| Feature | Source | Pourquoi |
|---------|--------|----------|
| `vma` | LIEUX | Vitesse = énergie cinétique |
| `catv` → `has_moto`, `has_poids_lourd` | VEHICULES | Vulnérabilité différente |
| `col` | CARACT | Frontale = très mortel |
| `agg` | CARACT | Rural = plus mortel (loin hôpital) |
| `catr` | LIEUX | Autoroute = haute vitesse |
| `lum` → `is_nuit` | CARACT | Nuit = plus grave |

## 🟡 Features moyennement prédictives
| Feature | Source | Pourquoi |
|---------|--------|----------|
| `obs` → obstacle fixe | VEHICULES | Arbre, poteau = mortel |
| `surf` | LIEUX | Surface glissante |
| `plan` → `is_virage` | LIEUX | Virages dangereux |
| `int` | CARACT | Type intersection |
| `atm` | CARACT | Météo dégradée |
| `nb_vehicules` | VEHICULES | Plus de véhicules = plus complexe |

## 🟢 Features utiles en complément
| Feature | Source | Pourquoi |
|---------|--------|----------|
| `heure` | CARACT | Heures de nuit |
| `jour_semaine` | CARACT | Weekend |
| `circ` | LIEUX | Bidirectionnelle = frontal possible |
| `prof` | LIEUX | Pente |
| `nb_usagers` | USAGERS | Plus de victimes potentielles |
| `has_pieton` | USAGERS | Piétons très vulnérables |

## ⚠️ Features à utiliser avec précaution (data leakage)
| Feature | Source | Problème |
|---------|--------|----------|
| `secu1/2/3` | USAGERS | Vérifié après accident |
| `choc` | VEHICULES | Constaté après |
| `manv` | VEHICULES | Reconstitué après |
| `place` | USAGERS | Position exacte après |
| `actp`, `locp` | USAGERS | Action piéton après |

---
# 8. Réponse à ta question : plusieurs use cases ?

## L'énoncé ne précise pas → C'est une opportunité !

Tu peux présenter **2-3 approches** dans ton projet :

### Approche 1 : "Aide aux secours" (principale)
- **Granularité** : par accident
- **Target** : accident mortel oui/non
- **Features** : uniquement celles disponibles en temps réel
- **Use case** : quand le SAMU reçoit un appel, prioriser

### Approche 2 : "Analyse épidémiologique" (bonus)
- **Granularité** : par usager
- **Target** : usager décédé oui/non  
- **Features** : toutes (y compris post-accident)
- **Use case** : comprendre les facteurs de mortalité

### Approche 3 : "Prévention" (bonus)
- **Granularité** : par accident
- **Target** : accident grave (tué + hospitalisé)
- **Features** : circonstances uniquement (pas collision)
- **Use case** : identifier les zones/conditions à risque

---

## Mon conseil

Fais **l'approche 1 en profondeur**, puis montre que tu as réfléchi aux autres.

Dans ta présentation :
> "J'ai choisi l'approche 'aide aux secours' car c'est actionnable en temps réel. 
> J'ai aussi exploré l'approche par usager pour l'analyse épidémiologique, 
> mais cela nécessite une validation spécifique (GroupKFold)."